In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [18]:
pd.read_sql(
    '''
    SELECT CHAS, AVG(CRIM) as avg_CRIM, AVG(ZN) as avg_ZN, AVG(INDUS) as avg_INDUS, AVG(NOX) as avg_NOX
  FROM boston GROUP BY CHAS;
    ''',
    con,
)

,CHAS,avg_CRIM,avg_ZN,avg_INDUS,avg_NOX
0,0.0,3.723215,11.634820,11.019193,0.551817
1,1.0,1.851670,7.714286,12.719143,0.593426


In [7]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT CRIM, COUNT() as "CRIM empty values", ZN, COUNT() as "ZN empty values", INDUS, COUNT() as "INDUS empty values", CHAS, COUNT() as "CHAS empty values", NOX, COUNT(*) as "NOX empty values" FROM boston WHERE CRIM IS NULL OR ZN IS NULL OR INDUS IS NULL OR CHAS IS NULL OR NOX IS NULL;
    ''',
    con,
)

,CRIM,CRIM empty values,ZN,ZN empty values,INDUS,INDUS empty values,CHAS,CHAS empty values,NOX,NOX empty values
0,None,0,None,0,None,0,None,0,None,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT CRIM, COUNT(DISTINCT CRIM) as "CRIM unique values", ZN, COUNT(DISTINCT ZN) as "ZN unique values", INDUS, COUNT(DISTINCT INDUS) as "INDUS unique values", CHAS, COUNT(DISTINCT CHAS) as "CHAS unique values", NOX, COUNT(DISTINCT NOX) as "NOX unique values" FROM boston;
    ''',
    con,
)

,CRIM,CRIM unique values,ZN,ZN unique values,INDUS,INDUS unique values,CHAS,CHAS unique values,NOX,NOX unique values
0,0.00632,504,18.0,26,2.31,76,0.0,2,0.538,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [10]:
tasks+=1
pd.read_sql(
    '''
    WITH t1 AS (
  SELECT
    column_name,
    MIN(column_value) AS min_value,
    AVG(column_value) AS avg_value
  FROM (
    SELECT
      'CRIM' AS column_name,
      CRIM AS column_value
    FROM boston
    UNION ALL
    SELECT
      'ZN' AS column_name,
      ZN AS column_value
    FROM boston
    UNION ALL
    SELECT
      'INDUS' AS column_name,
      INDUS AS column_value
    FROM boston
    UNION ALL
    SELECT
      'CHAS' AS column_name,
      CHAS AS column_value
    FROM boston
    UNION ALL
    SELECT
      'NOX' AS column_name,
      NOX AS column_value
    FROM boston
  )
  GROUP BY
    column_name
),
t2 AS (
  SELECT
    column_name,
    AVG(column_value) AS median_value
  FROM (
    SELECT
      column_name,
      column_value,
      ROW_NUMBER() OVER (PARTITION BY column_name ORDER BY column_value) AS row_num,
      COUNT(*) OVER (PARTITION BY column_name) AS total_rows
    FROM (
      SELECT
        column_name,
        column_value
      FROM (
        SELECT
          'CRIM' AS column_name,
          CRIM AS column_value
        FROM boston
        UNION ALL
        SELECT
          'ZN' AS column_name,
          ZN AS column_value
        FROM boston
        UNION ALL
        SELECT
          'INDUS' AS column_name,
          INDUS AS column_value
        FROM boston
        UNION ALL
        SELECT
          'CHAS' AS column_name,
          CHAS AS column_value
        FROM boston
        UNION ALL
        SELECT
          'NOX' AS column_name,
          NOX AS column_value
        FROM boston
      )
    )
  )
  WHERE
    row_num = total_rows / 2 + 1
    OR row_num = total_rows / 2
  GROUP BY
    column_name
)
SELECT
  t1.column_name
FROM
  t1
  JOIN t2 ON t1.column_name = t2.column_name
WHERE
  t1.avg_value = t2.median_value
  AND t1.min_value = t2.median_value;
    ''',
    con,
)

,column_name


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [9]:
tasks+=1
pd.read_sql(
    '''
    WITH most_expensive AS (
SELECT RM, MEDV
FROM boston
ORDER BY MEDV DESC
LIMIT 1
), cheapest_25 AS (
SELECT RM, MEDV
FROM boston
ORDER BY MEDV
LIMIT 25
), cheapest_50 AS (
SELECT RM, MEDV
FROM boston
ORDER BY MEDV
LIMIT 50
), cheapest_100 AS (
SELECT RM, MEDV
FROM boston
ORDER BY MEDV
LIMIT 100
), cheapest_200 AS (
SELECT RM, MEDV
FROM boston
ORDER BY MEDV
LIMIT 200
), cheapest_300 AS (
SELECT RM, MEDV
FROM boston
ORDER BY MEDV
LIMIT 300
)

SELECT 25 as "number of houses", avg(cheapest_25.RM) as "average rooms in cheapest 25", avg(most_expensive.RM) as "average rooms in most expensive", (avg(most_expensive.RM) - avg(cheapest_25.RM)) as "difference"
FROM most_expensive, cheapest_25
UNION
SELECT 50 as "number of houses", avg(cheapest_50.RM) as "average rooms in cheapest 50", avg(most_expensive.RM) as "average rooms in most expensive", (avg(most_expensive.RM) - avg(cheapest_50.RM)) as "difference"
FROM most_expensive, cheapest_50
UNION
SELECT 100 as "number of houses", avg(cheapest_100.RM) as "average rooms in cheapest 100", avg(most_expensive.RM) as "average rooms in most expensive", (avg(most_expensive.RM) - avg(cheapest_100.RM)) as "difference"
FROM most_expensive, cheapest_100
UNION
SELECT 200 as "number of houses", avg(cheapest_200.RM) as "average rooms in cheapest 200", avg(most_expensive.RM) as "average rooms in most expensive", (avg(most_expensive.RM) - avg(cheapest_200.RM)) as "difference"
FROM most_expensive, cheapest_200
UNION
SELECT 300 as "number of houses", avg(cheapest_300.RM) as "average rooms in cheapest 300", avg(most_expensive.RM) as "average rooms in most expensive", (avg(most_expensive.RM) - avg(cheapest_300.RM)) as "difference"
FROM most_expensive, cheapest_300;
    ''',
    con,
)

,number of houses,average rooms in cheapest 25,average rooms in most expensive,difference
0,25,5.747840,7.489,1.741160
1,50,5.753240,7.489,1.735760
2,100,5.887120,7.489,1.601880
3,200,5.911705,7.489,1.577295
4,300,5.972227,7.489,1.516773


Количество комнат влияет на цену, но незначительно

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT LSTAT, MEDV, RANK() OVER (ORDER BY LSTAT DESC)
FROM boston
ORDER BY MEDV DESC
    ''',
    con,
)

,LSTAT,MEDV,RANK() OVER (ORDER BY LSTAT DESC)
0,9.53,50.0,304
1,8.88,50.0,326
2,7.44,50.0,365
3,5.12,50.0,440
4,4.63,50.0,457
...,...,...,...
501,23.97,7.0,43
502,29.97,6.3,13
503,26.77,5.6,27
504,30.59,5.0,12


Чем выше стоимость - тем меньше голытьбы

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT
    (CASE WHEN CHAS = 0 THEN 'Not bordering the river' ELSE 'Bordering the river' END) as "Bordering",
    AVG(MEDV) as "Average Price"
    FROM boston
    GROUP BY CHAS;
    ''',
    con,
)

,Bordering,Average Price
0,Not bordering the river,22.093843
1,Bordering the river,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [20]:
tasks+=1
pd.read_sql(
    '''
    WITH cte AS (
  SELECT CHAS, AVG(CRIM) as avg_CRIM, AVG(ZN) as avg_ZN, AVG(INDUS) as avg_INDUS, AVG(NOX) as avg_NOX
  FROM boston
  GROUP BY CHAS
)
SELECT 
  (CASE WHEN cte.avg_CRIM > cte2.avg_CRIM THEN 'CRIM' ELSE NULL END) as column_name
  ,(CASE WHEN cte.avg_ZN > cte2.avg_ZN THEN 'ZN' ELSE NULL END) as column_name
  ,(CASE WHEN cte.avg_INDUS > cte2.avg_INDUS THEN 'INDUS' ELSE NULL END) as column_name
  ,(CASE WHEN cte.avg_NOX > cte2.avg_NOX THEN 'NOX' ELSE NULL END) as column_name
FROM cte
JOIN cte cte2
ON cte.CHAS <> cte2.CHAS AND cte.CHAS = 1
    ''',
    con,
)

,column_name,column_name,column_name,column_name
0,None,None,INDUS,NOX


Тяготение бизнеса к реке может быть объяснено более удобными транспортными путями и как следствие, повышение уровня оксидов азота

Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX).8.  Напишите прослеживается между ними взаимосвязь

In [48]:
tasks+=1
pd.read_sql(
    '''
  WITH cte AS (
  SELECT 
    INDUS, 
    NOX,
    NTILE(10) OVER (ORDER BY INDUS) AS indus_tile, 
    NTILE(10) OVER (ORDER BY NOX) AS nox_tile
  FROM boston
)
SELECT 
  10 * indus_tile AS percentile, 
  AVG(INDUS) AS indus_value, 
  AVG(NOX) AS nox_value
FROM cte
GROUP BY indus_tile
ORDER BY percentile;
        ''',
    con,
)

,percentile,indus_value,nox_value
0,10,2.016078,0.438378
1,20,3.645294,0.494129
2,30,5.246275,0.448235
3,40,6.471765,0.470839
4,50,8.256078,0.531510
5,60,10.749216,0.515176
6,70,17.245800,0.564920
7,80,18.100000,0.684880
8,90,18.307200,0.714620
9,100,21.938800,0.692860


In [ ]:
### В целом наличие неторгового(мб производственного) бизнеса связано с повышение уровня оксидов азота в воздухе

In [ ]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')